# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [308]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [309]:
RANDOM_STATE = 110

train_data = pd.read_csv("../../data/trim_train_data.csv")
test_data = pd.read_csv("../../data/trim_test_data.csv")

---

In [310]:
# target 열을 임시로 분리
target_train = train_data['target']
target_test = test_data['target']

# 모든 값이 NaN인 열 제거
train_data = train_data.dropna(axis=1, how='all')
test_data = test_data.dropna(axis=1, how='all')

# target 열을 다시 결합
train_data['target'] = target_train
test_data['target'] = target_test

In [311]:
# Wip Line 열 제거
wip_line_columns = train_data.filter(like='Wip Line').columns

train_data.drop(columns=wip_line_columns, inplace=True)
test_data.drop(columns=wip_line_columns, inplace=True)

In [312]:
# Process Desc 열 제거
Process_Desc_col = train_data.filter(like='Process Desc').columns

train_data.drop(columns=Process_Desc_col, inplace=True)
test_data.drop(columns=Process_Desc_col, inplace=True)

In [313]:
# Equipment로 시작하는 열 필터링
Equipment_col = train_data.filter(like='Equipment').columns
Equipment_col2 = test_data.filter(like='Equipment').columns

new_train = train_data.filter(items=Equipment_col)
new_test = test_data.filter(items=Equipment_col2)

# Equipment_same_num 파생변수 생성
def determine_equipment_same_num(row):
    if (row['Equipment_Dam'] == 'Dam dispenser #1' and row['Equipment_AutoClave'] == 'Auto Clave Out' and 
        row['Equipment_Fill1'] == 'Fill1 dispenser #1' and row['Equipment_Fill2'] == 'Fill2 dispenser #1') or \
       (row['Equipment_Dam'] == 'Dam dispenser #2' and row['Equipment_AutoClave'] == 'Auto Clave Out' and 
        row['Equipment_Fill1'] == 'Fill1 dispenser #2' and row['Equipment_Fill2'] == 'Fill2 dispenser #2'):
        return 1
    else:
        return 0

train_data['Equipment_same_num'] = new_train.apply(determine_equipment_same_num, axis=1)
test_data['Equipment_same_num'] = new_test.apply(determine_equipment_same_num, axis=1)

train_data = train_data.drop(columns=['Equipment_Dam', 'Equipment_AutoClave', 'Equipment_Fill1', 'Equipment_Fill2'])
test_data = test_data.drop(columns=['Equipment_Dam', 'Equipment_AutoClave', 'Equipment_Fill1', 'Equipment_Fill2'])


In [314]:
# Model.Suffix_Dam의 이름을 Model.Suffix로 변경
train_data = train_data.rename(columns={'Model.Suffix_Dam': 'Model.Suffix'})
test_data = test_data.rename(columns={'Model.Suffix_Dam': 'Model.Suffix'})

# Model.Suffix_AutoClave, Model.Suffix_Fill1, Model.Suffix_Fill2 열 드롭
train_data = train_data.drop(columns=['Model.Suffix_AutoClave', 'Model.Suffix_Fill1', 'Model.Suffix_Fill2'])
test_data = test_data.drop(columns=['Model.Suffix_AutoClave', 'Model.Suffix_Fill1', 'Model.Suffix_Fill2'])

In [315]:
# Model.Suffix_Dam의 이름을 Model.Suffix로 변경
train_data = train_data.rename(columns={'Workorder_Dam': 'Workorder'})
test_data = test_data.rename(columns={'Workorder_Dam': 'Workorder'})

# Model.Suffix_AutoClave, Model.Suffix_Fill1, Model.Suffix_Fill2 열 드롭
train_data = train_data.drop(columns=['Workorder_AutoClave', 'Workorder_Fill1', 'Workorder_Fill2'])
test_data = test_data.drop(columns=['Workorder_AutoClave', 'Workorder_Fill1', 'Workorder_Fill2'])

In [316]:
# Insp. Seq No 열 제거
Insp_Seq_No_col = train_data.filter(like='Insp. Seq No').columns

train_data.drop(columns=Insp_Seq_No_col, inplace=True)
test_data.drop(columns=Insp_Seq_No_col, inplace=True)

In [317]:
# Insp Judge Code 열 제거
Insp_Judge_Code_col = train_data.filter(like='Insp Judge Code').columns

train_data.drop(columns=Insp_Judge_Code_col, inplace=True)
test_data.drop(columns=Insp_Judge_Code_col, inplace=True)

In [318]:
# 값의 종류가 1개이고 결측값이 없는 열을 제거하는 함수
def drop_single_value_columns(df):
    cols_to_drop = [col for col in df.columns if col != 'target' and df[col].nunique() == 1 and df[col].isnull().sum() == 0]
    df_dropped = df.drop(columns=cols_to_drop)
    return df_dropped, cols_to_drop

# train_data와 test_data에서 해당 열 제거 및 삭제된 열 이름과 개수 출력
train_data, train_cols_dropped = drop_single_value_columns(train_data)
test_data, test_cols_dropped = drop_single_value_columns(test_data)

# print("삭제된 train_data 열 이름:", train_cols_dropped)
# print("삭제된 train_data 열 개수:", len(train_cols_dropped))

# print("삭제된 test_data 열 이름:", test_cols_dropped)
# print("삭제된 test_data 열 개수:", len(test_cols_dropped))

In [319]:
train_data.info()
print('---')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 119 entries, Model.Suffix to Equipment_same_num
dtypes: float64(58), int64(52), object(9)
memory usage: 36.8+ MB
---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Columns: 120 entries, Set ID to Equipment_same_num
dtypes: float64(94), int64(17), object(9)
memory usage: 15.9+ MB


---

개별 공정별 데이터 전처리

## Dam

## AutoClave

In [320]:
# 변수명 변경
train_data = train_data.rename(columns={'1st Pressure 1st Pressure Unit Time_AutoClave': '1st Pressure Unit Time_AutoClave'})
test_data = test_data.rename(columns={'1st Pressure 1st Pressure Unit Time_AutoClave': '1st Pressure Unit Time_AutoClave'})

In [321]:
train_data['2nd_Pressure_Greater_Than_0.6'] = train_data['2nd Pressure Collect Result_AutoClave'].apply(lambda x: 1 if x >= 0.6 else 0)
test_data['2nd_Pressure_Greater_Than_0.6'] = test_data['2nd Pressure Collect Result_AutoClave'].apply(lambda x: 1 if x >= 0.6 else 0)

In [322]:
train_data['3rd_Pressure_Greater_Than_0.7'] = train_data['3rd Pressure Collect Result_AutoClave'].apply(lambda x: 1 if x >= 0.7 else 0)
test_data['3rd_Pressure_Greater_Than_0.7'] = test_data['3rd Pressure Collect Result_AutoClave'].apply(lambda x: 1 if x >= 0.7 else 0)

In [323]:
train_data.info()
print('---')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 121 entries, Model.Suffix to 3rd_Pressure_Greater_Than_0.7
dtypes: float64(58), int64(54), object(9)
memory usage: 37.4+ MB
---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Columns: 122 entries, Set ID to 3rd_Pressure_Greater_Than_0.7
dtypes: float64(94), int64(19), object(9)
memory usage: 16.2+ MB


## Fill1

## Fill2

---

In [324]:
# 각 변수별로 결측값이 존재하는지 확인하는 코드
missing_values = train_data.isnull().sum()

# 결측값이 존재하는 변수와 그 개수 출력
missing_values = missing_values[missing_values > 0]
print(missing_values)

# 결측값이 존재하는 변수명을 리스트에 담기
missing_columns = missing_values.index.tolist()
# print("결측값이 존재하는 변수명:", missing_columns)

HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam      29213
WorkMode Collect Result_Dam                                24059
GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave       29213
GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave          29213
HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1    29213
WorkMode Collect Result_Fill1                              24059
HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2    29213
WorkMode Collect Result_Fill2                              24059
dtype: int64


In [325]:
# 결측값이 존재하는 변수들의 value_counts 계산
for column in missing_columns:
    print(f"Column: {column}")
    print(train_data[column].value_counts())
    print("\n")

Column: HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam
OK    11293
Name: HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam, dtype: int64


Column: WorkMode Collect Result_Dam
7.0    16447
Name: WorkMode Collect Result_Dam, dtype: int64


Column: GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave
OK    11293
Name: GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave, dtype: int64


Column: GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave
OK    11293
Name: GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave, dtype: int64


Column: HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1
OK    11293
Name: HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1, dtype: int64


Column: WorkMode Collect Result_Fill1
7.0    16447
Name: WorkMode Collect Result_Fill1, dtype: int64


Column: HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2
OK    11293
Name: HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2, dtype: int64


Column: WorkMode Collect Result_Fill2
0.0    16447

In [326]:
# "OK" 값을 가진 변수를 찾고 제거
columns_to_drop = [column for column in missing_columns if train_data[column].apply(lambda x: np.any(x == "OK")).any()]

# 드롭할 변수명 출력 (한 줄에 2개씩)
print("드롭할 변수명:")
for i in range(0, len(columns_to_drop), 2):
    print(columns_to_drop[i:i+2])

# 변수 드롭
train_data = train_data.drop(columns=columns_to_drop)
test_data = test_data.drop(columns=columns_to_drop)

드롭할 변수명:
['HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam', 'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave']
['GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave', 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1']
['HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2']


In [327]:
# 각 변수별로 결측값이 존재하는지 확인하는 코드
missing_values = train_data.isnull().sum()

# 결측값이 존재하는 변수와 그 개수 출력
missing_values = missing_values[missing_values > 0]
print(missing_values)

# 결측값이 존재하는 변수명을 리스트에 담기
missing_columns = missing_values.index.tolist()
# print("결측값이 존재하는 변수명:", missing_columns)

WorkMode Collect Result_Dam      24059
WorkMode Collect Result_Fill1    24059
WorkMode Collect Result_Fill2    24059
dtype: int64


In [328]:
# 그룹화할 변수들
groupby_columns = [
    "WorkMode Collect Result_Dam"
    , "WorkMode Collect Result_Fill1"
    , "WorkMode Collect Result_Fill2"
    ]

# 그룹화하여 target 변수 값의 비율을 계산
grouped = train_data.groupby(groupby_columns)["target"].value_counts(normalize=True).unstack().fillna(0)

# 결과 출력
print(grouped)

target                                                                                   AbNormal  \
WorkMode Collect Result_Dam WorkMode Collect Result_Fill1 WorkMode Collect Result_Fill2             
7.0                         7.0                           0.0                            0.073326   

target                                                                                     Normal  
WorkMode Collect Result_Dam WorkMode Collect Result_Fill1 WorkMode Collect Result_Fill2            
7.0                         7.0                           0.0                            0.926674  


In [329]:
# WorkMode Collect Result_Dam의 이름을 WorkMode Collect Result로 변경
train_data = train_data.rename(columns={'WorkMode Collect Result_Dam': 'WorkMode Collect Result'})
test_data = test_data.rename(columns={'WorkMode Collect Result_Dam': 'WorkMode Collect Result'})

# WorkMode Collect Result_Fill1, WorkMode Collect Result_Fill2 열 드롭
train_data = train_data.drop(columns=['WorkMode Collect Result_Fill1', 'WorkMode Collect Result_Fill2'])
test_data = test_data.drop(columns=['WorkMode Collect Result_Fill1', 'WorkMode Collect Result_Fill2'])

In [330]:
# WorkMode Collect Result 열의 값이 7인 행을 1로 변경
train_data['WorkMode Collect Result'] = train_data['WorkMode Collect Result'].replace(7, 1)
test_data['WorkMode Collect Result'] = test_data['WorkMode Collect Result'].replace(7, 1)

# WorkMode Collect Result 열의 결측값을 0으로 채움
train_data['WorkMode Collect Result'] = train_data['WorkMode Collect Result'].fillna(0)
test_data['WorkMode Collect Result'] = test_data['WorkMode Collect Result'].fillna(0)

In [331]:
test_data['WorkMode Collect Result'].value_counts()

0.0    10349
1.0     7012
Name: WorkMode Collect Result, dtype: int64

In [332]:
# 각 변수별로 결측값이 존재하는지 확인하는 코드
missing_values = train_data.isnull().sum()

# 결측값이 존재하는 변수와 그 개수 출력
missing_values = missing_values[missing_values > 0]
print(missing_values)

# 결측값이 존재하는 변수명을 리스트에 담기
missing_columns = missing_values.index.tolist()
print("결측값이 존재하는 변수명:", missing_columns)

Series([], dtype: int64)
결측값이 존재하는 변수명: []


In [333]:
# 각 변수별로 결측값이 존재하는지 확인하는 코드
missing_values = test_data.isnull().sum()

# 결측값이 존재하는 변수와 그 개수 출력
missing_values = missing_values[missing_values > 0]
print(missing_values)

# 결측값이 존재하는 변수명을 리스트에 담기
missing_columns = missing_values.index.tolist()
print("결측값이 존재하는 변수명:", missing_columns)

target    17361
dtype: int64
결측값이 존재하는 변수명: ['target']


### 문자형(object) 변수 -> 수치형 변환


In [334]:
train_data['target'].value_counts()

Normal      38156
AbNormal     2350
Name: target, dtype: int64

In [335]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 114 entries, Model.Suffix to 3rd_Pressure_Greater_Than_0.7
dtypes: float64(56), int64(54), object(4)
memory usage: 35.2+ MB


In [336]:
print(test_data['target'].dtype)

float64


타겟 인코딩

In [337]:
# 'target' 열의 변수 타입을 object로 변경
# -> test 데이터는 float64 타입으로 되어있음 
test_data['target'] = test_data['target'].astype('object')

# object 타입의 변수 출력
train_object_columns = train_data.select_dtypes(include=['object']).columns
test_object_columns = test_data.select_dtypes(include=['object']).columns

print(train_object_columns, f" train_object_columns 갯수 : {len(train_object_columns)}")
print(test_object_columns, f" test_object_columns 갯수 : {len(test_object_columns)}")

# 각 object 변수의 고유 값 개수 출력
print("\nTrain Data:")
for col in train_object_columns:
    unique_count = train_data[col].nunique()
    print(f"{col} unique 값 갯수: {unique_count}")

print("\nTest Data:")
for col in test_object_columns:
    unique_count = test_data[col].nunique()
    print(f"{col} unique 값 갯수: {unique_count}")

Index(['Model.Suffix', 'Workorder', 'Chamber Temp. Judge Value_AutoClave',
       'target'],
      dtype='object')  train_object_columns 갯수 : 4
Index(['Set ID', 'Model.Suffix', 'Workorder',
       'Chamber Temp. Judge Value_AutoClave', 'target'],
      dtype='object')  test_object_columns 갯수 : 5

Train Data:
Model.Suffix unique 값 갯수: 7
Workorder unique 값 갯수: 663
Chamber Temp. Judge Value_AutoClave unique 값 갯수: 2
target unique 값 갯수: 2

Test Data:
Set ID unique 값 갯수: 17361
Model.Suffix unique 값 갯수: 7
Workorder unique 값 갯수: 662
Chamber Temp. Judge Value_AutoClave unique 값 갯수: 2
target unique 값 갯수: 0


In [338]:
# 필요한 라이브러리 임포트
import pandas as pd
import category_encoders as ce

# 타겟 변수와 범주형 변수 지정
## Target Encoding의 smoothing 파라미터는 default로 auto로 설정되어 있음
target = 'target'  # 타겟 변수 이름으로 변경
categorical_columns = [
    'Model.Suffix',
    'Workorder',
    'Chamber Temp. Judge Value_AutoClave'
]  # 범주형 변수 이름으로 변경

# 타겟 값을 숫자로 변환
target_mapping = {'Normal': 0, 'AbNormal': 1}
train_data[target] = train_data[target].map(target_mapping)
test_data[target] = test_data[target].map(target_mapping)

# 열이 존재하는지 확인
missing_columns = [col for col in categorical_columns if col not in train_data.columns]
if missing_columns:
    raise ValueError(f"train_data에 다음 열이 존재하지 않습니다: {missing_columns}")

# 타겟 인코더 생성 및 학습
encoder = ce.TargetEncoder(cols=categorical_columns)
train_data = encoder.fit_transform(train_data, train_data[target])

# Set ID 열을 별도로 저장
set_id = test_data['Set ID']

# 테스트 데이터 인코딩 (Set ID 열 제외)
test_data = test_data.drop(columns=['Set ID'])
test_data = encoder.transform(test_data)

# Set ID 열을 맨 앞에 추가
test_data.insert(0, 'Set ID', set_id)

# categorical_columns에 해당하는 열의 데이터 값만 확인
print(train_data[categorical_columns].head(3))
print(test_data[categorical_columns].head(3))

# 역 매핑 딕셔너리 생성
reverse_target_mapping = {v: k for k, v in target_mapping.items()}

# 타겟 값을 원래대로 변환
train_data[target] = train_data[target].map(reverse_target_mapping)
test_data[target] = test_data[target].map(reverse_target_mapping)

print("--- train_data ---")

# 변환된 타겟 값 확인
print(train_data[[target]].value_counts())

   Model.Suffix  Workorder  Chamber Temp. Judge Value_AutoClave
0      0.049336   0.158385                             0.058361
1      0.049336   0.015314                             0.058361
2      0.056712   0.009534                             0.058361
   Model.Suffix  Workorder  Chamber Temp. Judge Value_AutoClave
0      0.056712   0.091912                             0.058361
1      0.056712   0.024247                             0.058361
2      0.056712   0.091463                             0.058361
--- train_data ---
target  
Normal      38156
AbNormal     2350
dtype: int64


In [339]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 114 entries, Model.Suffix to 3rd_Pressure_Greater_Than_0.7
dtypes: float64(59), int64(54), object(1)
memory usage: 35.2+ MB


In [340]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Columns: 115 entries, Set ID to 3rd_Pressure_Greater_Than_0.7
dtypes: float64(94), int64(19), object(2)
memory usage: 15.2+ MB


In [341]:
# 필요한 라이브러리 임포트
from imblearn.over_sampling import SMOTE

# SMOTE를 사용하여 오버 샘플링 수행
smote = SMOTE(random_state=RANDOM_STATE)
X_train = train_data.drop(columns=['target'])
y_train = train_data['target']
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# 오버 샘플링된 데이터셋을 DataFrame으로 변환
train_data = pd.concat([pd.DataFrame(X_resampled, columns=X_train.columns), pd.Series(y_resampled, name='target')], axis=1)

In [342]:
train_data['target'].value_counts()

Normal      38156
AbNormal    38156
Name: target, dtype: int64

## 3. 모델 학습


### 모델 정의


In [343]:
model = RandomForestClassifier(random_state=RANDOM_STATE)

### 모델 학습


In [344]:
features = []

for col in train_data.columns:
    try:
        train_data[col] = train_data[col].astype(int)
        features.append(col)
    except:
        continue

train_x = train_data[features]
train_y = train_data["target"]

model.fit(train_x, train_y)

RandomForestClassifier(random_state=110)

## 4. 제출하기


### 테스트 데이터 예측


테스트 데이터 불러오기


In [345]:
# test_data에 존재하는 열만 선택
valid_features = [col for col in features if col in test_data.columns]

df_test_x = test_data.loc[:, valid_features]

for col in df_test_x.columns:
    try:
        df_test_x.loc[:, col] = df_test_x[col].astype(int)
    except:
        continue

In [346]:
# 학습 시 사용된 피처 이름을 저장합니다.
train_features = model.feature_names_in_

# 예측 시 동일한 피처를 사용하도록 데이터프레임을 조정합니다.
df_test_x = df_test_x.reindex(columns=train_features, fill_value=0)

# 예측을 수행합니다.
test_pred = model.predict(df_test_x)
test_pred

array(['Normal', 'Normal', 'Normal', ..., 'AbNormal', 'Normal', 'Normal'],
      dtype=object)

### 제출 파일 작성


In [347]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

In [348]:
df_sub['target'].value_counts()

Normal      15887
AbNormal     1474
Name: target, dtype: int64

In [349]:
df_sub.head()

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal


**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
